In [94]:
%load_ext autoreload
%autoreload 2

import MDAnalysis as mda
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis as HBA
from MDAnalysis.analysis import rdf
from scipy.signal import find_peaks
import numpy as np
from ase.visualize import view
from ase.io import read
import numpy as np
from pathlib import Path
import os 

def pdb_to_coords(pdb_path, n_atoms_per_res):
    lines = [_ for _ in open(pdb_path).readlines() if _[0] == "A"]
    coords = []
    total = 0

    for n in n_atoms_per_res:
        tmp_lines = lines[total:total + n]
        tmp_lines = [f"{_.split()[2][:1]} {_[31:38]} {_[39:46]} {_[47:54]}" for _ in tmp_lines]
        coords.append(tmp_lines)
        total += n

    return coords


def coords_to_xyz(coords):
    """Take a list of lists coordinates and return a string in xyz format"""
    out = []
    n_atoms = sum([len(_) for _ in coords])
    out.append(f"{n_atoms}\n\n")

    for i, _ in enumerate(coords):
        for line in _:
            atom_type, x, y, z = line.split()
            out.append(f"{atom_type} {x} {y} {z}\n")
    return "".join(out)


def count_same(l):
    o = [1]
    for i, _ in enumerate(l):
        if not i == 0:
            if l[i] == l[i-1]:
                o[-1] = o[-1] + 1
            else:
                o.append(1)
        else:
            o.append(1)
        
    return o


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:
dcdspath = "/home/boittier/pcbach/charmmions/"
Ndcds = len(list(Path(dcdspath).glob("step5*dcd")))
dcd = ["/home/boittier/pcbach/charmmions/step5_{}.dcd".format(i+1) for i in range(Ndcds)]
dcd = [_ for _ in dcd if os.path.exists(_) and (os.stat(_).st_size > 0)]
print(dcd)
psf = "/home/boittier/pcbach/charmmions/step3_pbcsetup.psf"

['/home/boittier/pcbach/charmmions/step5_1.dcd', '/home/boittier/pcbach/charmmions/step5_2.dcd', '/home/boittier/pcbach/charmmions/step5_3.dcd', '/home/boittier/pcbach/charmmions/step5_4.dcd']


In [97]:
u = mda.Universe(psf, dcd)

/home/boittier/miniconda3/envs/pycharmm/lib/python3.8/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [98]:
u.trajectory

<ChainReader containing step5_1.dcd and 3 more with 40 frames of 2536 atoms>

In [99]:
# import nglview as nv

In [100]:
# w = nv.show_mdanalysis(u)
# w

In [153]:
from MDAnalysis import Writer

Natoms = 44
maxTries = 1000
dr = 0.05
resname = "POT"

def save_N_atoms(Natoms, resname, 
                 maxTries=100, 
                 dr=0.05,
                 defaultR=4.750,
                 verbose=False,
                ):
    selectStringTypes = f"(byres resname {resname})"
    selectStringDist = "(byres point {rx} {ry} {rz} {r}) or resid {resid}"
    
    # outfile for an xyz trajectory
    xyz_out_name = f"{Natoms}_{resname}.xyz"
    W = Writer(xyz_out_name, Natoms)  
    
    xyz_counter = 0
    n_atoms = []
    pdb_paths = []
    for i, t in enumerate(u.trajectory):
        r = np.random.rand(3) * 1
        res_sel1 = u.select_atoms(selectStringTypes, updating=False, periodic=False)
        # loop thru the residues of the required type
        for j in [_.resid for _ in res_sel1]:
            counter = 0
            n_current = 0
            #  try to avoid an index error
            try:
                j = j - len(res_sel1)
                #  initialize the selection
                res = u.select_atoms(selectStringTypes, updating=True, 
                                     periodic=False)
                #  gather the appropriate data
                resname = res_sel1[j].resname
                rx = res_sel1[j].position[0]
                ry = res_sel1[j].position[1]
                rz = res_sel1[j].position[2]

                #  while loop to adjust r until the conditions are met
                while n_current != Natoms and counter < maxTries:
                    # format the distance string
                    _selectStringDist = selectStringDist.format(
                        rx=rx, ry=ry, rz=rz, r=defaultR, resid=j)
                    #  make the selection again
                    res = u.select_atoms(_selectStringDist, 
                                         updating=False, periodic=True)
                    #  change the radius
                    if len(res) < Natoms:
                        defaultR += dr
                    else:
                        defaultR -= dr
                    counter += 1
                    n_current = len(res)

                #  exiting the while loop
                #  ....printing for debugging
                if verbose:
                    if counter >= maxTries:
                        print("conditions not met:", n_current)
                        
                n_current = len(res)
                # check for success
                if n_current == Natoms:
                    if verbose:
                        print("success: ", n_current, defaultR, counter)
                    xyz_counter += 1
                    resids = [_.resid for _ in res]
                    # paths
                    name = f"test_coords/{Natoms}_{xyz_counter}_{resname}_{i}_{j}"
                    pdb_path = name+".pdb"
                    xyz_path = name+".xyz"
                    #  write 
                    res.write(pdb_path)
                    res.write(xyz_path)
                    #  count the number of atoms per residue
                    natom_res = count_same(resids)
                    #  get the coords list
                    coords = pdb_to_coords(pdb_path, natom_res)
                    xyz_str = coords_to_xyz(coords)
                    n_atoms.append(float(xyz_str[:3]))
                    pdb_paths.append(pdb_path)
                    #  write to the xyz writer
                    W.write(res)
                    
            #  handle errors
            except IndexError as e:
                if verbose:
                    print(e)
                    
    # close the file
    W.close()
    
    # if the file has no atoms, delete it
    with open(xyz_out_name) as f:
        if len(f.readlines()) < 2:
            if verbose:
                print("deleting ", xyz_out_name)
            os.system(f"rm {xyz_out_name}")
    

    

In [155]:
from itertools import product as product

natoms_list = [35, 41, 44, 47, 50, 53, 56, 59, 62]
natoms_list = range(30, 70)
res_list = [
    "POT",
    "CLA"]

for n, r in product(natoms_list, res_list):
    print(r, n)
    save_N_atoms(n, r, verbose=True)

POT 30
2 <AtomGroup [<Atom 2533: POT of type POT of resname POT, resid 1 and segid IONS>, <Atom 2534: POT of type POT of resname POT, resid 2 and segid IONS>]>
j= 1
conditions not met: 31
31 30
j= 2
conditions not met: 31
31 30
2 <AtomGroup [<Atom 2533: POT of type POT of resname POT, resid 1 and segid IONS>, <Atom 2534: POT of type POT of resname POT, resid 2 and segid IONS>]>
j= 1
conditions not met: 28
28 30
j= 2
conditions not met: 28
28 30
2 <AtomGroup [<Atom 2533: POT of type POT of resname POT, resid 1 and segid IONS>, <Atom 2534: POT of type POT of resname POT, resid 2 and segid IONS>]>
j= 1
conditions not met: 31
31 30
j= 2
conditions not met: 28
28 30
2 <AtomGroup [<Atom 2533: POT of type POT of resname POT, resid 1 and segid IONS>, <Atom 2534: POT of type POT of resname POT, resid 2 and segid IONS>]>
j= 1
conditions not met: 28
28 30
j= 2
conditions not met: 31
31 30
2 <AtomGroup [<Atom 2533: POT of type POT of resname POT, resid 1 and segid IONS>, <Atom 2534: POT of type PO

In [110]:
# atoms = read("62_POT.xyz")
# view(atoms)#, viewer="x3d")

In [10]:
print(pdb_path)
atoms = read(pdb_paths[6])
view(atoms)#, viewer="x3d")

test_coords/CLA_9_3.pdb


In [12]:
print(pdb_path)
atoms = Atoms([read(x) for x in pdb_paths])
view(atoms, viewer="x3d")

test_coords/CLA_9_3.pdb


NameError: name 'Atoms' is not defined

In [69]:
pdb_paths

['test_coords/POT_0_1.pdb',
 'test_coords/CLA_0_2.pdb',
 'test_coords/CLA_0_3.pdb',
 'test_coords/POT_1_1.pdb',
 'test_coords/CLA_1_2.pdb',
 'test_coords/CLA_1_3.pdb',
 'test_coords/POT_2_1.pdb',
 'test_coords/CLA_2_2.pdb',
 'test_coords/CLA_2_3.pdb',
 'test_coords/POT_3_1.pdb',
 'test_coords/CLA_3_2.pdb',
 'test_coords/CLA_3_3.pdb',
 'test_coords/POT_4_1.pdb',
 'test_coords/CLA_4_2.pdb',
 'test_coords/CLA_4_3.pdb',
 'test_coords/POT_5_1.pdb',
 'test_coords/CLA_5_2.pdb',
 'test_coords/CLA_5_3.pdb',
 'test_coords/POT_6_1.pdb',
 'test_coords/CLA_6_2.pdb',
 'test_coords/CLA_6_3.pdb',
 'test_coords/POT_7_1.pdb',
 'test_coords/CLA_7_2.pdb',
 'test_coords/CLA_7_3.pdb',
 'test_coords/POT_8_1.pdb',
 'test_coords/CLA_8_2.pdb',
 'test_coords/CLA_8_3.pdb',
 'test_coords/POT_9_1.pdb',
 'test_coords/CLA_9_2.pdb',
 'test_coords/CLA_9_3.pdb']